In [ ]:
msfile = 'ngc3621.ms'
prefix='ngc3621'
target='NGC3621'
g_cal='1117-248'
b_cal='0408-65'
f_cal='0408-65'
#ref_ant='m018'
ref_ant='m003'

### [Optional] W correction

### Basics calibration steps

In [ ]:
clearcal(msfile)

### Convert correlation coefficients to absolute flux densities

In [ ]:
log_S = -0.9790 + 3.3662*np.log10(1284.) -1.1216*np.log10(1284.)**2 + 0.0861*np.log10(1284.)**3
setjy(vis=msfile, field=f_cal, scalebychan=True, standard='manual', fluxdensity=[10**log_S,0,0,0])

### Delay calibration using the flux cal

In [ ]:
ktable = prefix + '.K'

In [ ]:
rmtables(ktable)

In [ ]:
gaincal(vis=msfile,
        caltable=ktable,
        field=f_cal,
        gaintype='K',
        solint='inf',
        refant=ref_ant,
        combine='scan',
        solnorm=False,
        minsnr=3.0,
        gaintable=[])

In [ ]:
plotcal(caltable=ktable,
        xaxis='antenna',
        yaxis='delay')

### Through away calibration using the flux cal

In [ ]:
gtable0 = prefix + '.G0'

In [ ]:
rmtables(gtable0)

In [ ]:
gaincal(vis=msfile,
        caltable=gtable0,
        field=f_cal,
#        solint='int',
        solint='inf',
        refant=ref_ant,
        gaintype='G',
        solnorm=True,
        calmode='p',
        minsnr=3.0,
        gaintable=[ktable])

In [ ]:
plotcal(caltable=gtable0,
        xaxis='time',
        yaxis='phase',
        plotrange = [-1, -1, -180, 180])

### Solve for bandpass response using the flux cal

In [ ]:
btable = prefix + '.B'

In [ ]:
rmtables(btable)

In [ ]:
bandpass(vis=msfile,
         caltable=btable,
         field=b_cal,
         refant=ref_ant,
         solnorm=True,
         combine='scan',
         solint='inf',
         bandtype='B',
         minsnr=3.0,
         gaintable=[gtable0, ktable])

In [ ]:
plotcal(caltable=btable,
        xaxis='chan',
        yaxis='phase',
        field=b_cal,
        plotrange = [-1, -1, -180, 180])

### Complex gain calibration

In [ ]:
gtable = prefix + '.G'

In [ ]:
rmtables(gtable)

In [ ]:
gaincal(vis=msfile,
        caltable=gtable,
        field=f_cal,
        solint='int',
        refant=ref_ant,
        combine='spw',
        gaintype='G',
        calmode='ap',
        minsnr=1.0,
        solnorm=False,
        gaintable=[btable, ktable])

In [ ]:
gaincal(vis=msfile,
        caltable=gtable,
        field=g_cal,
        solint='int',
        refant=ref_ant,
        combine='spw',
        gaintype='G',
        calmode='ap',
        minsnr=1.0,
        solnorm=False,
        append=True,
        gaintable=[btable, ktable])

In [ ]:
plotcal(caltable=gtable,
        xaxis='time',
        yaxis='phase',
        field=f_cal+','+g_cal,
        plotrange=[-1, -1, -180, 180])

### Flux calibration

In [ ]:
ftable = prefix + '.flux'

In [ ]:
rmtables(ftable)

In [ ]:
fluxscale(vis=msfile,
          caltable=gtable,
          fluxtable=ftable,
          reference=f_cal,
          transfer=g_cal)

### Apply calibration separately to each source

In [ ]:
applycal(vis=msfile,
         field=f_cal,
         gaintable=[ftable, btable, ktable],
         gainfield=[f_cal, f_cal, f_cal],
         interp=['', 'nearest', ''],
         calwt=False,
         applymode='calflag')

In [ ]:
plotms(vis=msfile,
       xaxis='time',
       yaxis='amp',
       ydatacolumn='corrected',
       correlation='XX,YY',
       field=f_cal,
       coloraxis='corr',
       averagedata=True,
       avgbaseline=True,
       avgchannel='4096')

In [ ]:
applycal(vis=msfile,
         field=g_cal,
         gaintable=[ftable, btable, ktable],
         gainfield=[g_cal, b_cal, f_cal],
         interp=['', 'nearest', ''],
         calwt=False,
         applymode='calflag')

In [ ]:
plotms(vis=msfile,
       xaxis='time',
       yaxis='amp',
       ydatacolumn='corrected',
       correlation='XX,YY',
       field=g_cal,
       coloraxis='corr',
       averagedata=True,
       avgbaseline=True,
       avgchannel='4096')

### Verify efficiency of calibration and flagging